#  利用stan生成GP数据

用stan模型生成Gaussion Process 数据
用于产生stan模型，并保存为文件：

1. gen_exp.plk: 指数相关函数

2. gen_matern32.pkl: mater3/2相关函数

In [1]:
import pystan
%load_ext jupyterstan

import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

from stan_utils import *

## 构建stan模型

### 指数相关函数模型

$$ C(d) = \alpha^{2} exp(-\frac{d}{\rho}) $$

In [6]:
%%stan gen_exp
functions {
  matrix cov_exp(vector[] x, real alpha, real rho) {
    int N=size(x);
    matrix[N, N] cov;
    for (i in 1:N){
      for (j in i:N){
          cov[i,j] = square(alpha) * exp(-1*distance(x[i], x[j]) / rho);
          cov[j,i] = cov[i, j];
      }
    }    
    return cov;
  }
}
data {
  int<lower=1> N;
  vector[2] x[N];
  
  real<lower=0> alpha;
  real<lower=0> rho;
  real<lower=0> sigma;
}

transformed data {
  matrix[N, N] cov = cov_exp(x, alpha, rho) + diag_matrix(rep_vector(1e-10, N));
  
  matrix[N, N] L_cov = cholesky_decompose(cov);
}

parameters {}
model {}

generated quantities {
  vector[N] f = multi_normal_cholesky_rng(rep_vector(0,N), L_cov);
  vector[N] y;
  for (n in 1:N){
    y[n] = normal_rng(f[n], sigma);
  }
}

INFO:pystan:COMPILING THE C++ CODE FOR MODEL gen_exp_573dd760c5907e4eb43fba0b386fd0cf NOW.


Creating pystan model & assigning it to variable name "gen_exp".
Stan options:
 {'model_name': 'gen_exp', 'include_paths': None, 'boost_lib': None, 'eigen_lib': None, 'verbose': False, 'obfuscate_model_name': True}
StanModel now available as variable "gen_exp"!
Compilation took a minute.


In [18]:
StanModel_cache(gen_exp, 'gen_exp')

Model cached as:gen_exp.pkl


### matern32模型

$$ C(d) = \alpha^{2}(1+\frac{d}{\rho}) exp(-\frac{d}{\rho}) $$

In [20]:
%%stan gen_matern32
functions {
  matrix cov_matern32(vector[] x, real alpha, real rho) {
    int N=size(x);
    matrix[N, N] cov;
    for (i in 1:N){
      for (j in i:N){
          cov[i,j] = square(alpha) * (1 + distance(x[i],x[j])/rho) * exp(-1*distance(x[i], x[j]) / rho);
          cov[j,i] = cov[i, j];
      }
    }    
    return cov;
  }
}
data {
  int<lower=1> N;
  vector[2] x[N];
  
  real<lower=0> alpha;
  real<lower=0> rho;
  real<lower=0> sigma;
}

transformed data {
  matrix[N, N] cov = cov_matern32(x, alpha, rho) + diag_matrix(rep_vector(1e-10, N));
  
  matrix[N, N] L_cov = cholesky_decompose(cov);
}

parameters {}
model {}

generated quantities {
  vector[N] f = multi_normal_cholesky_rng(rep_vector(0,N), L_cov);
  vector[N] y;
  for (n in 1:N){
    y[n] = normal_rng(f[n], sigma);
  }
}

INFO:pystan:COMPILING THE C++ CODE FOR MODEL gen_matern32_7d446a1ecb6fb7776885eaf3a5461ad5 NOW.


Creating pystan model & assigning it to variable name "gen_matern32".
Stan options:
 {'model_name': 'gen_matern32', 'include_paths': None, 'boost_lib': None, 'eigen_lib': None, 'verbose': False, 'obfuscate_model_name': True}
StanModel now available as variable "gen_matern32"!
Compilation took a minute.


In [21]:
StanModel_cache(gen_matern32, 'gen_matern32')

Model cached as:gen_matern32.pkl


### matern52模型